In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

# Define dictionaries for column names and types
PROCEDURE_COLUMNS = {f"PROCEDIMIENTO{i}": str for i in range(1, 31)}
SERVICE_COLUMNS = {
    "SERVICIOINGRESO": 0,
    "SERVICIOTRASLADO1": 1,
    "SERVICIOTRASLADO2": 2,
    "SERVICIOTRASLADO3": 3,
    "SERVICIOTRASLADO4": 4,
    "SERVICIOTRASLADO5": 5,
    "SERVICIOTRASLADO6": 6,
    "SERVICIOTRASLADO7": 7,
    "SERVICIOTRASLADO8": 8,
    "SERVICIOTRASLADO9": 9,
    "SERVICIOALTA": 10,
}
DATE_COLUMNS = {
    "FECHA_INGRESO": 0,
    "FECHATRASLADO1": 1,
    "FECHATRASLADO2": 2,
    "FECHATRASLADO3": 3,
    "FECHATRASLADO4": 4,
    "FECHATRASLADO5": 5,
    "FECHATRASLADO6": 6,
    "FECHATRASLADO7": 7,
    "FECHATRASLADO8": 8,
    "FECHATRASLADO9": 9,
    "FECHAALTA": 10,
}
BED_TYPES = {
    "AREA MEDICA ADULTO CUIDADOS BASICOS": "Basicos",
    "AREA MEDICA ADULTO CUIDADOS MEDIOS": "Medias",
    "AREA MEDICO-QUIRURGICO CUIDADOS MEDIOS": "Medias",
    "AREA MÉDICA": "Medias",
    "AREA QUIRÚRGICA": "Medias",
    "CIRUGÍA CARDIOVASCULAR": "Medias",
    "UNIDAD DE CUIDADOS INTENSIVOS (UCI) (INDIFERENCIADO)": "UCI",
    "UNIDAD DE CUIDADOS INTENSIVOS ADULTO": "UCI",
    "UNIDAD DE CUIDADOS INTENSIVOS CARDIOLOGÍA": "UCI",
    "UNIDAD DE RECUPERACIÓN DE PABELLONES (CENTRAL Y CMA)": "AMBULATORIO",
    "UNIDAD DE TRATAMIENTO INTERMEDIO (UTI) (INDIFERENCIADO) ADULTO": "UTI",
    "UNIDAD DE TRATAMIENTO INTERMEDIO CARDIOVASCULAR": "UTI",
    "UNIDAD DE TRATAMIENTO INTERMEDIO CIRUGÍA ADULTO": "UTI",
    "UNIDAD DE TRATAMIENTO INTERMEDIO MEDICINA ADULTO": "UTI",
    "UNIDAD DE TRATAMIENTOS INTERMEDIOS MEDICINA": "UTI",
}

# Define relevant diagnoses
RELEVANT_DIAGNOSES = [
    "C33",
    "C34.0",
    "C34.1",
    "C34.2",
    "C34.3",
    "C38.1",
    "C38.4",
    "C45.0",
    "C78.0",
    "C78.2",
    "D14.3",
    "D38.1",
    "E84.8",
    "I05.1",
    "I08.0",
    "I08.1",
    "I34.0",
    "I35.0",
    "I35.1",
    "I35.2",
    "I42.0",
    "I45.6",
    "I47.2",
    "I49.5",
    "I71.0",
    "I71.2",
    "J39.8",
    "J47",
    "J67.9",
    "J84.1",
    "J84.8",
    "J84.9",
    "J86.0",
    "J86.9",
    "J90",
    "J93.1",
    "J95.5",
    "J98.0",
    "M34.8",
    "Q21.1",
    "Q23.1",
    "Q67.6",
    "T82.0",
    "T82.1",
    "Z45.0",
]

# Load data
df_procesada = pd.read_csv("../data/processed/df_procesada.csv", sep=";", dtype=PROCEDURE_COLUMNS)

/var/folders/wy/2slj_t7s2vz19v2h2926qb340000gn/T/ipykernel_15819/754381329.py:105: DtypeWarning: Columns (24,25,26,27,28,29,30,31,32,33,37,41,43,47,51,75,76,77,78,79,80,81,82,83,84,85,86,87,123) have mixed types. Specify dtype option on import or set low_memory=False.
  df_procesada = pd.read_csv("../data/processed/df_procesada.csv", sep=";", dtype=PROCEDURE_COLUMNS)


In [19]:
# Filter for Torax hospital
df_torax = df_procesada.query("COD_HOSPITAL == 112103").reset_index()

# Melt service and date columns
service_long = pd.melt(
    df_torax,
    id_vars=["index", "DIAGNOSTICO1", "ANIO_EGRESO", "IR_29301_SEVERIDAD", "CIP_ENCRIPTADO"],
    value_vars=SERVICE_COLUMNS,
    var_name="service_type",
    value_name="service",
)
date_long = pd.melt(
    df_torax,
    id_vars=["index"],
    value_vars=DATE_COLUMNS,
    var_name="date_type",
    value_name="date",
)

# Merge service and date data
patient_journey = pd.merge(date_long, service_long, how="inner", left_index=True, right_index=True)
patient_journey = patient_journey.drop(columns=["index_y"])
patient_journey = patient_journey.rename(columns={"index_x": "index"})

# Convert service type to categorical
patient_journey["service_type"] = pd.Categorical(
    patient_journey["service_type"], categories=SERVICE_COLUMNS.keys(), ordered=True
)

# Sort and clean data
patient_journey = (
    patient_journey.sort_values(["index", "service_type"]).dropna().reset_index(drop=True)
)
patient_journey["date"] = pd.to_datetime(patient_journey["date"])

# Calculate length of stay
patient_journey["length_of_stay"] = patient_journey.groupby("index")["date"].diff().shift(-1)

# Fix negative and zero length of stay
negative_indices = patient_journey[patient_journey["length_of_stay"] < pd.Timedelta(0)].index + 1
patient_journey.loc[negative_indices, "date"] += pd.offsets.DateOffset(years=1)
patient_journey["length_of_stay"] = patient_journey.groupby("index")["date"].diff().shift(-1)

zero_indices = patient_journey[patient_journey["length_of_stay"] == pd.Timedelta(0)].index
patient_journey.loc[zero_indices, "length_of_stay"] += pd.Timedelta(days=1)

# Map bed types
patient_journey["service"] = patient_journey["service"].replace(BED_TYPES)

# Summarize length of stay by diagnosis, bed type, and year
summary = (
    patient_journey.groupby(["ANIO_EGRESO", "DIAGNOSTICO1", "IR_29301_SEVERIDAD", "service"])[
        "length_of_stay"
    ]
    .sum()
    .unstack()
)
summary_pct = summary.apply(lambda x: x / summary.sum(axis=1), axis=0)
summary_total = pd.concat([summary, summary_pct], axis=1).reset_index()

# Filter for relevant diagnoses
summary_relevant = summary_total.query("DIAGNOSTICO1.isin(@RELEVANT_DIAGNOSES)")

In [15]:
# Save to Excel
with pd.ExcelWriter("../data/interim/resumen_ocupacion_por_diagnostico.xlsx") as writer:
    summary_relevant.to_excel(writer, sheet_name="mas_relevantes", index=False)
    summary_total.to_excel(writer, sheet_name="todos", index=False)

In [20]:
patient_journey

,index,date_type,date,DIAGNOSTICO1,ANIO_EGRESO,IR_29301_SEVERIDAD,CIP_ENCRIPTADO,service_type,service,length_of_stay
0,3946,FECHA_INGRESO,2019-01-14,I25.1,2019.0,1.0,1137216.0,SERVICIOINGRESO,Medias,3 days
1,3946,FECHAALTA,2019-01-17,I25.1,2019.0,1.0,1137216.0,SERVICIOALTA,Medias,NaT
2,9385,FECHA_INGRESO,2019-08-26,Z51.1,2019.0,3.0,1084360.0,SERVICIOINGRESO,Medias,1 days
3,9385,FECHAALTA,2019-08-26,Z51.1,2019.0,3.0,1084360.0,SERVICIOALTA,Medias,NaT
4,9442,FECHA_INGRESO,2019-01-09,Z51.1,2019.0,2.0,822388.0,SERVICIOINGRESO,Medias,1 days
...,...,...,...,...,...,...,...,...,...,...
37403,2749044,FECHATRASLADO2,2021-10-29,I25.1,2021.0,3.0,73993832.0,SERVICIOTRASLADO2,UCI,11 days
37404,2749044,FECHATRASLADO3,2021-11-09,I25.1,2021.0,3.0,73993832.0,SERVICIOTRASLADO3,Medias,1 days
37405,2749044,FECHATRASLADO4,2021-11-10,I25.1,2021.0,3.0,73993832.0,SERVICIOTRASLADO4,UCI,2 days
37406,2749044,FECHATRASLADO5,2021-11-12,I25.1,2021.0,3.0,73993832.0,SERVICIOTRASLADO5,UTI,4 days


In [22]:
df_torax[["DIAGNOSTICO1", "CIP_ENCRIPTADO"]].value_counts()

DIAGNOSTICO1  CIP_ENCRIPTADO
Z51.1         1084360.0         26
Z29.8         984077.0          24
Z29.1         336626.0          17
Z51.1         1331104.0         17
Z29.1         1433519.0         17
                                ..
I26.9         1391061.0          1
              1419784.0          1
              68599635.0         1
              72564239.0         1
Z54.8         247976.0           1
Name: count, Length: 11281, dtype: int64

In [25]:
df_torax.query("DIAGNOSTICO1 == 'E84.8'")["CIP_ENCRIPTADO"].value_counts()

CIP_ENCRIPTADO
67020837.0    6
504615.0      5
575037.0      5
292467.0      5
67538884.0    4
1234704.0     4
1312263.0     3
73577836.0    3
77847077.0    3
69384631.0    3
140108.0      3
70969924.0    3
826226.0      3
587708.0      3
606192.0      3
1284432.0     3
417078.0      2
96218367.0    2
68374265.0    2
71103866.0    2
68352950.0    2
894397.0      2
72581218.0    2
1368884.0     2
388402.0      2
769581.0      2
130614.0      2
71004150.0    2
71008001.0    1
69426880.0    1
96519794.0    1
568189.0      1
67812784.0    1
79413176.0    1
67076327.0    1
72078668.0    1
67135576.0    1
67047156.0    1
79660257.0    1
67037361.0    1
96921973.0    1
67153696.0    1
398110.0      1
67154931.0    1
512466.0      1
796134.0      1
800600.0      1
381371.0      1
1266727.0     1
1033733.0     1
623539.0      1
626982.0      1
68392935.0    1
23440.0       1
477707.0      1
1028010.0     1
836197.0      1
1459078.0     1
67032013.0    1
75479000.0    1
Name: count, dtype: int64